.env file can include the following environment variables to set:

OPENAI_API_KEY,
TRANSFORMERS_CACHE,
HUGGINGFACE_HUB_CACHE,
HF_HOME,
XDG_CACHE_HOME,

Example:
```
OPENAI_API_KEY=123
TRANSFORMERS_CACHE=/yun.hy/.cache/huggingface
```

In [1]:
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path=Path("/scratch/yun.hy/llm-meta-analysis/.env"))

True

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import hf_olmo

model = AutoModelForCausalLM.from_pretrained(
            "allenai/OLMo-7B-Instruct", device_map="auto"
        )

# 2048
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-7B-Instruct")

/home/yun.hy/.conda/envs/llm-meta-analysis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yun.hy/.conda/envs/llm-meta-analysis/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.33s/it]


In [8]:
def generate(model, tokenizer, prompt, max_new_tokens=50): # chat version seems to do better despite format not being easy to parse
    chat = [
        {"role": "user", "content": input},
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        result = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(result[0, inputs.input_ids.shape[1]:], skip_special_tokens=True)

In [9]:
def output_type_prompt1(outcome: str):
    return f'Is the outcome of {outcome} from a randomized controlled trial a binary or continuous type? Output "b" if the outcome type is "binary", "c" if the outcome type is "continuous", or "x" if there is insufficient information to make any inference. Do not provide any explanation and just provide one letter answer.'

def output_type_prompt2(abstract_and_results_xml: str, outcome: str):
    return f'Article: {abstract_and_results_xml}. Based on the article, is the outcome of {outcome} from a randomized controlled trial a binary or continuous type? Output "b" if the outcome type is "binary", "c" if the outcome type is "continuous", or "x" if there is insufficient information to make any inference. Do not provide any explanation and just provide one letter answer.'

In [10]:
def binary_outcomes_prompt(abstract_and_results_xml: str, intervention: str, comparator: str, outcome: str):
    return f'''
        Article: {abstract_and_results_xml}

        Based on the given trial article, what is the 2x2 contingency table in YAML format for the following Intervention, Comparator, and Outcome? If any of the numerical information is unavailable or not extractable or not easy to calculate, please say "x". Only provide YAML response. Do not provide explanation.
        The YAML format should include the fields "events" and "group_size" for only "intervention" and "comparator" but not "outcome". Example: 
        intervention:
            events: NUMBER
            group_size: NUMBER	
        comparator:
            events: NUMBER
            group_size: NUMBER

        Intervention: {intervention}
        Comparator: {comparator}
        Outcome: {outcome}
        '''

In [11]:
def continuous_outcomes_prompt(abstract_and_results_xml: str, intervention: str, comparator: str, outcome: str):
    return f'''
        Article: {abstract_and_results_xml}

        Based on the given trial article, what is table of mean outcome and standard deviation in YAML format for the following Intervention, Comparator, and Outcome. Include the total size of each group for Intervention and Comparator. If any of the numerical information is unavailable or not extractable or not easy to calculate, please say "x". Only provide YAML response. Do not provide an explanation.
        The YAML format should include the fields "mean", "standard_deviation", and "group_size" for only "intervention" and "comparator" but not "outcome". Example: 
        intervention:
            mean: NUMBER
            standard_deviation: NUMBER
            group_size: NUMBER
        comparator:
            mean: NUMBER
            standard_deviation: NUMBER
            group_size: NUMBER

        Intervention: {intervention}
        Comparator: {comparator}
        Outcome: {outcome}
        '''

### Binary Outcome Type

In [12]:
prompt = output_type_prompt1("mortality")
generate(model, tokenizer, prompt, max_new_tokens=50)

'Is the outcome of mortality from a randomized controlled trial a binary or continuous type? Output "b" if the outcome type is "binary", "c" if the outcome type is "continuous", or "x" if there is insufficient information to make any inference. Do not provide any explanation and just provide one letter answer.\n\nMortality is a binary outcome.\nb'

### Continuous Outcome Type

In [13]:
article = """
    <abstract><sec><title>OBJECTIVE</title><p>Moderate alcohol consumption is associated with reduced incidence of type 2 diabetes and cardiovascular mortality and increases adiponectin concentrations, but effects might differ according to sex and beverage consumed.</p></sec><sec><title>RESEARCH DESIGN AND METHODS</title><p>A total of 72 healthy individuals (22&#x02013;56 years) were enrolled in this randomized controlled crossover trial. After washout, two interventions for 3 weeks followed: ethanol (concentration 12.5%), beer (5.6%), or red wine (12.5%) equivalent to 30 g ethanol/day for men and 20 g/day for women or the same de-alcoholized beverages or water. Adiponectin was measured by sandwich enzyme-linked immunosorbent assay.</p></sec><sec><title>RESULTS</title><p>Among women, adiponectin significantly increased after consuming red wine (29.8%, <italic>P</italic> &#x0003c; 0.05) and increased among men after ethanol solution (17.4%, <italic>P</italic> &#x0003c; 0.05) and consuming beer (16.1%, <italic>P</italic> &#x0003c; 0.05). De-alcoholized beverages had no substantial effect on adiponectin concentrations.</p></sec><sec><title>CONCLUSIONS</title><p>Moderate amounts of ethanol-containing beverages increased adiponectin concentrations, but sex-specific effects might depend on type of beverage consumed.</p></sec></abstract><sec sec-type="results"><title>RESULTS</title><p>Compliance was excellent according to self-report and counting of empty bottles returned. One participant was excluded because of protocol violation. Baseline characteristics including age, BMI, and liver enzymes did not differ within sexes and between intervention groups. Adiponectin concentrations were substantially higher among females (means 7.6&#x02013;8.8 &#x003bc;g/ml) than males (means 4.8&#x02013;6.3 &#x003bc;g/ml) but also did not differ between the intervention groups within each sex. Among females, adiponectin significantly increased after intervention with red wine (29.8%, <italic>P</italic> &#x0003c; 0.05) and increased among men after ethanol solution (17.4%, <italic>P</italic> &#x0003c; 0.05) and beer consumption (16.1%, <italic>P</italic> &#x0003c; 0.04). De-alcoholized beverages had no substantial effect on adiponectin concentrations (<xref ref-type="fig" rid="F1">Fig. 1</xref>).</p><fig id="F1" position="float"><label>Figure 1</label><caption><p>Mean percent change with standard deviation of adiponectin concentrations from baseline after intervention with beer (B+), de-alcoholized beer (B&#x02212;), red wine (RW+), de-alcoholized red wine (RW&#x02212;), enthanol solution (EtOH), and water (W) (*<italic>P</italic> &#x0003c; 0.05).</p></caption><graphic xlink:href="zdc0060975660001" /></fig></sec>
"""

prompt = output_type_prompt2(article, "Adiponectin in women")
generate(model, tokenizer, prompt, max_new_tokens=50)

'Article: \n    <abstract><sec><title>OBJECTIVE</title><p>Moderate alcohol consumption is associated with reduced incidence of type 2 diabetes and cardiovascular mortality and increases adiponectin concentrations, but effects might differ according to sex and beverage consumed.</p></sec><sec><title>RESEARCH DESIGN AND METHODS</title><p>A total of 72 healthy individuals (22&#x02013;56 years) were enrolled in this randomized controlled crossover trial. After washout, two interventions for 3 weeks followed: ethanol (concentration 12.5%), beer (5.6%), or red wine (12.5%) equivalent to 30 g ethanol/day for men and 20 g/day for women or the same de-alcoholized beverages or water. Adiponectin was measured by sandwich enzyme-linked immunosorbent assay.</p></sec><sec><title>RESULTS</title><p>Among women, adiponectin significantly increased after consuming red wine (29.8%, <italic>P</italic> &#x0003c; 0.05) and increased among men after ethanol solution (17.4%, <italic>P</italic> &#x0003c; 0.05)

In [14]:
prompt = output_type_prompt2(article, "Adiponectin in women")
generate(model, tokenizer, prompt, max_new_tokens=50)

'Article: \n    <abstract><sec><title>OBJECTIVE</title><p>Moderate alcohol consumption is associated with reduced incidence of type 2 diabetes and cardiovascular mortality and increases adiponectin concentrations, but effects might differ according to sex and beverage consumed.</p></sec><sec><title>RESEARCH DESIGN AND METHODS</title><p>A total of 72 healthy individuals (22&#x02013;56 years) were enrolled in this randomized controlled crossover trial. After washout, two interventions for 3 weeks followed: ethanol (concentration 12.5%), beer (5.6%), or red wine (12.5%) equivalent to 30 g ethanol/day for men and 20 g/day for women or the same de-alcoholized beverages or water. Adiponectin was measured by sandwich enzyme-linked immunosorbent assay.</p></sec><sec><title>RESULTS</title><p>Among women, adiponectin significantly increased after consuming red wine (29.8%, <italic>P</italic> &#x0003c; 0.05) and increased among men after ethanol solution (17.4%, <italic>P</italic> &#x0003c; 0.05)

### Binary Outcomes 2x2 Table

In [9]:
article = """
    <abstract><sec><title>Background:</title><p>Anderson-Hynes dismembered pyeloplasty is the gold standard therapeutic approach to ureteropelvic junction obstruction (UPJO). Use of a drainage method to protect the suture line from leakage is a matter of controversy.</p></sec><sec><title>Objectives:</title><p>We have compared the surgical outcome of Anderson-Hynes dismembered pyeloplasty for UPJO repair, with or without internal stenting.</p></sec><sec><title>Patients and Methods:</title><p>Eighty-two patients with UPJO were evaluated from 1996 to 2002. Complicated or emergent cases were excluded. Classic standard dismembered pyeloplasty was performed. Internal drainage, with a double j catheter, was performed in several patients, randomly. Another drain was also placed in the retroperitoneal space. The follow-up of patients was planned weekly, with patient visits and urine analysis and intravenous pyelography (IVP) and diethylene-triamine-pentaacetate (DTPA) scan after one month.</p></sec><sec><title>Results:</title><p>The study group consisted of 51 male and 31 female patients, who were mostly in the age range of 20 - 40 years. Comparing the two techniques of pyeloplasty with or without internal drainage, there was no significant difference between groups regarding extravasation and anastomosis complications, such as leakage, stenosis, urinoma formation or evidence of obstruction on postoperative IVP or DTPA scan. However, a higher incidence of catheter related urinary symptoms and flank pain was reported among those with internal stent.</p></sec><sec><title>Conclusions:</title><p>Pyeloplasty, with adequate spatulation, hemostasis and a watertight anastomosis, represents the mainstay of successful pyeloplasty and there may be no significant benefit for urethral stenting, especially in non-complicated cases.</p></sec></abstract><sec id="sec92058"><title>4. Results</title><p>Double stenting was used in 30 cases and pyeloplasty without internal drainage was performed in 52 patients. Both groups were similar in age and sex distribution. The most common symptom was severe flank pain, which was present in 69% of patients. History of urinary tract infection was detected in 47% and 51% reported associated gastrointestinal symptoms. Pyuria was seen in 58% and only 36% of patients had positive urine culture, with <italic>Escherichia coli</italic> (<italic>E. coli</italic>) as the most common pathogen (60%). The preoperative sonographic evaluation and IVP results are shown in <xref ref-type="table" rid="tbl23577">Table 1</xref>. Intraoperative assessments showed ureteropelvic junction segmental atresia in 45 cases and aberrant vessels in 25 patients. The postoperative course was compared in two groups of patients, with and without internal drainage (<xref ref-type="table" rid="tbl23578">Table 2</xref>). Among patients with extravasation in DTPA scan in the stent free group, retrograde DJ stent was inserted in four cases that had unacceptable drainage due to inflammation. One patient had distal ureter obstruction because of distal ureter stone, that was treated by Trans-ureteral lithotripsy (TULP) and stent support. Three other cases had no anatomical abnormality, and they were followed and extravasation resolved spontaneously at control IVP after one month. Removal of DJ stent was planned in one month after operation. However, eight cases missed the time and late stent removal was accompanied with several complications, such as stone formation on the tip of catheter in the bladder of a patient that needed TULP before stent removal.</p><table-wrap id="tbl23577" orientation="portrait" position="float"><label>Table 1.</label><caption><title> Results of Preoperative Ultrasonography and Intravenous Pyelography Evaluation <sup><xref ref-type="table-fn" rid="fn21576">a</xref>, <xref ref-type="table-fn" rid="fn21577">b</xref></sup></title></caption><table frame="hsides" rules="groups"><thead><tr><th style="text-align: left;" rowspan="1" colspan="1">Imaging Method</th><th rowspan="1" colspan="1">Right Kidney</th><th rowspan="1" colspan="1">Left Kidney</th></tr></thead><tbody><tr><td rowspan="1" colspan="1"><bold>Sonography</bold></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Severe hydronephrosis</td><td style="text-align: center;" rowspan="1" colspan="1">28 (34.6)</td><td style="text-align: center;" rowspan="1" colspan="1">35 (43.2)</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Moderate hydronephrosis</td><td style="text-align: center;" rowspan="1" colspan="1">8 (9.9)</td><td style="text-align: center;" rowspan="1" colspan="1">9 (11.1)</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Mild hydronephrosis</td><td style="text-align: center;" rowspan="1" colspan="1">6 (7.4)</td><td style="text-align: center;" rowspan="1" colspan="1">3 (3.7)</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Without hydronephrosis</td><td style="text-align: center;" rowspan="1" colspan="1">39 (48)</td><td style="text-align: center;" rowspan="1" colspan="1">34 (42)</td></tr><tr><td rowspan="1" colspan="1"><bold>IVP</bold></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">No secretion</td><td style="text-align: center;" rowspan="1" colspan="1">6 (7.3)</td><td style="text-align: center;" rowspan="1" colspan="1">9 (11)</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Delayed secretion</td><td style="text-align: center;" rowspan="1" colspan="1">33 (40.2)</td><td style="text-align: center;" rowspan="1" colspan="1">38 (46.3)</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Normal secretion</td><td style="text-align: center;" rowspan="1" colspan="1">43 (52.4)</td><td style="text-align: center;" rowspan="1" colspan="1">35 (42.7)</td></tr></tbody></table><table-wrap-foot><fn id="fn21576"><p><sup>a</sup>Abbreviation: IVP, intravenous pyelography</p></fn><fn id="fn21577"><p><sup>b</sup>Data are presented as No. (%).</p></fn></table-wrap-foot></table-wrap><table-wrap id="tbl23578" orientation="portrait" position="float"><label>Table 2.</label><caption><title> Comparing Postoperative Course Between Patients who Underwent Pyeloplasty With or Without Internal Drainage <sup><xref ref-type="table-fn" rid="fn21578">a</xref>, <xref ref-type="table-fn" rid="fn21579">b</xref></sup></title></caption><table frame="hsides" rules="groups"><thead><tr><th style="text-align: left;" rowspan="1" colspan="1">Post-Operative Course</th><th rowspan="1" colspan="1">With Stent (n = 30)</th><th rowspan="1" colspan="1">Without Stent (n = 52)</th><th rowspan="1" colspan="1">P Value</th></tr></thead><tbody><tr><td rowspan="1" colspan="1"><bold>Drain removal</bold></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">In 48 hours</td><td style="text-align: center;" rowspan="1" colspan="1">12 (40)</td><td style="text-align: center;" rowspan="1" colspan="1">17 (32.7)</td><td style="text-align: center;" rowspan="1" colspan="1">0.583</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">After 2 days</td><td style="text-align: center;" rowspan="1" colspan="1">18 (60)</td><td style="text-align: center;" rowspan="1" colspan="1">34 (65.4)</td><td style="text-align: center;" rowspan="1" colspan="1">0.632</td></tr><tr><td rowspan="1" colspan="1"><bold>Mean hospital stay, d</bold></td><td style="text-align: center;" rowspan="1" colspan="1">36 &#x000b1; 1.74</td><td style="text-align: center;" rowspan="1" colspan="1">3.4 &#x000b1; 1.30</td><td style="text-align: center;" rowspan="1" colspan="1">0.841</td></tr><tr><td rowspan="1" colspan="1"><bold>Urine analysis </bold><sup><bold /><bold><xref ref-type="table-fn" rid="fn21580">c</xref></bold></sup></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Hematuria</td><td style="text-align: center;" rowspan="1" colspan="1">19/27</td><td style="text-align: center;" rowspan="1" colspan="1">32/32</td><td style="text-align: center;" rowspan="1" colspan="1">0.001</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Pyuria</td><td style="text-align: center;" rowspan="1" colspan="1">12/27</td><td style="text-align: center;" rowspan="1" colspan="1">20/32</td><td style="text-align: center;" rowspan="1" colspan="1">0.008</td></tr><tr><td rowspan="1" colspan="1"><bold>Post-operative IVP (one month later)</bold></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Extravasation</td><td style="text-align: center;" rowspan="1" colspan="1">2 (6.7)</td><td style="text-align: center;" rowspan="1" colspan="1">8 (15.3)</td><td style="text-align: center;" rowspan="1" colspan="1">0.209</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Delayed secretion</td><td style="text-align: center;" rowspan="1" colspan="1">19 (63.3)</td><td style="text-align: center;" rowspan="1" colspan="1">25 (48.1)</td><td style="text-align: center;" rowspan="1" colspan="1">0.124</td></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Visible ureter</td><td style="text-align: center;" rowspan="1" colspan="1">18 (60)</td><td style="text-align: center;" rowspan="1" colspan="1">41 (78.8)</td><td style="text-align: center;" rowspan="1" colspan="1">0.633</td></tr><tr><td rowspan="1" colspan="1"><bold>DTPA scan</bold><sup><bold /><bold><xref ref-type="table-fn" rid="fn21581">d</xref></bold></sup></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">Partial obstruction</td><td style="text-align: center;" rowspan="1" colspan="1">8/20</td><td style="text-align: center;" rowspan="1" colspan="1">13/44</td><td style="text-align: center;" rowspan="1" colspan="1">0.716</td></tr><tr><td rowspan="1" colspan="1"><bold>Urinary symptoms </bold></td><td style="text-align: center;" rowspan="1" colspan="1">24 (80)</td><td style="text-align: center;" rowspan="1" colspan="1">2 (3.8)</td><td style="text-align: center;" rowspan="1" colspan="1">0.005</td></tr><tr><td rowspan="1" colspan="1"><bold>Urinoma</bold></td><td style="text-align: center;" rowspan="1" colspan="1">0 (0)</td><td style="text-align: center;" rowspan="1" colspan="1">1 (1.9)</td><td style="text-align: center;" rowspan="1" colspan="1">0.324</td></tr><tr><td rowspan="1" colspan="1"><bold>Recurrence of stenosis</bold></td><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /><td style="text-align: center;" rowspan="1" colspan="1" /></tr><tr><td style="padding-left: 20pt;" rowspan="1" colspan="1">reoperation</td><td style="text-align: center;" rowspan="1" colspan="1">0 (0)</td><td style="text-align: center;" rowspan="1" colspan="1">1 (1.9)</td><td style="text-align: center;" rowspan="1" colspan="1">0.324</td></tr><tr><td rowspan="1" colspan="1"><bold>Stent remnants</bold></td><td style="text-align: center;" rowspan="1" colspan="1">2 (6.7)</td><td style="text-align: center;" rowspan="1" colspan="1">0 (0)</td><td style="text-align: center;" rowspan="1" colspan="1">0.041</td></tr><tr><td rowspan="1" colspan="1"><bold>Flank pain at voiding</bold></td><td style="text-align: center;" rowspan="1" colspan="1">10 (33.3)</td><td style="text-align: center;" rowspan="1" colspan="1">0 (0)</td><td style="text-align: center;" rowspan="1" colspan="1">0.001</td></tr></tbody></table><table-wrap-foot><fn id="fn21578"><p><sup>a</sup>Abbreviations: IVP, intravenous pyelography; DTPA, diethylene-triamine-pentaacetate.</p></fn><fn id="fn21579"><p><sup>b</sup>Data are presented as No. (%).</p></fn><fn id="fn21580"><p><sup>c</sup>Urine analysis was done in 27 patients of the stent group and 32 cases of the stent free group.</p></fn><fn id="fn21581"><p><sup>d</sup>DTPA scan was done in 20 patients of stent group and 32 cases of stent free group.</p></fn></table-wrap-foot></table-wrap></sec>
"""
prompt = binary_outcomes_prompt(article, "internal drainage group", "stentless cases", "ureteropelvic junction obstruction")
generate(model, tokenizer, prompt, max_new_tokens=100)

'\n        Article: \n    <abstract><sec><title>Background:</title><p>Anderson-Hynes dismembered pyeloplasty is the gold standard therapeutic approach to ureteropelvic junction obstruction (UPJO). Use of a drainage method to protect the suture line from leakage is a matter of controversy.</p></sec><sec><title>Objectives:</title><p>We have compared the surgical outcome of Anderson-Hynes dismembered pyeloplasty for UPJO repair, with or without internal stenting.</p></sec><sec><title>Patients and Methods:</title><p>Eighty-two patients with UPJO were evaluated from 1996 to 2002. Complicated or emergent cases were excluded. Classic standard dismembered pyeloplasty was performed. Internal drainage, with a double j catheter, was performed in several patients, randomly. Another drain was also placed in the retroperitoneal space. The follow-up of patients was planned weekly, with patient visits and urine analysis and intravenous pyelography (IVP) and diethylene-triamine-pentaacetate (DTPA) scan

### Continuous Outcomes

In [10]:
article = """
    <abstract><p>In order to assess the effects of juice feedings during acute diarrhea a double-blind, randomized study was performed in 90 children, mean age of 10 &#x000b1; 4.28 months. Thirty patients with acute diarrhea were fed twice-daily 15 ml/kg of Apple Juice (AJ), 30 received White Grape Juice (WGJ), and 30 were given colored and flavored water (WA) as part of their age appropriate dietary intake. The duration and severity of diarrhea were the main endpoint variables of the study performed in a metabolic unit. The patients were similar among the 3 groups, had diarrhea for 50&#x02013;64 hours prior to admission, and were dehydrated when admitted to the unit for study. Half of the patients in each group were well nourished and the others had mild to moderate degrees of malnutrition. Rotavirus infection was the agent causing the illness in 63% of the patients. The infants fed juice ingested 14&#x02013;17% more calories than those given WA, (those receiving AJ and WGJ ingested 95 and 98 Calories/Kg/d respectively) whereas those receiving WA consumed 81 cal/kg/d). The increased energy intake was not at the expense of other foods or milk formula. The mean body weight gain was greater among patients receiving WGJ (+ 50.7 gm) as compared with the patients in the AJ group (+ 18.3 gm) or the patients fed WA (- 0.7 gm) (p = 0.08). The duration of the illness was longer in the infants fed juice as compared with those given WA (p = 0.006), the mean +/- SD duration in hours was 49.4 &#x000b1; 32.6, 47.5 &#x000b1; 38.9 and 26.5 &#x000b1; 27.4 in patients fed AJ, WGJ and WA respectively. All patients improved while ingesting juice and none of them developed persistent diarrhea; most recovered within 50 hours of the beginning of treatment and less than one fourth had diarrhea longer than 96 hours in the unit. The fecal losses were also increased among the juice fed patients (p = 0.001); the mean &#x000b1; SD fecal excretion in g/kg/h was 3.94 &#x000b1; 2.35, 3.59 &#x000b1; 2.35, and 2.19 &#x000b1; 1.63 in AJ, WGJ and WA respectively. The stool output was highest during the first day of treatment among all the patients, though those fed AJ had the highest volume of fecal losses and those who received WA had the lowest stool excretion. After the first day of treatment the differences in fecal excretion were not significant. The ability to tolerate carbohydrates during the illness and immediately after recovery was similar among the 3 groups of patients. Intake of juices with different fructose/glucose ratios and osmolarities resulted in more fecal losses and more prolonged diarrhea as compared with water feedings, but the patients given juice ingested more calories and gained more weight, particularly among those being fed the juice with equimolar concentrations of fructose and glucose.</p></abstract><sec><title>Results</title><p>The clinical characteristics and the laboratory data of the patients on admission are shown in tables <xref ref-type="table" rid="T2">2</xref> and <xref ref-type="table" rid="T3">3</xref>. The patients in each of the 3 groups were similar in age, duration and severity of diarrhea, presence of fever and vomiting. Also there were no differences in the proportion of patients who received breast feedings or in their nutritional status. Over 46% of patients studied in each group, were well nourished, more than 33% showed mild body weight deficits (&#x0003c;1 SD) and the others had mild to moderate malnutrition (&#x0003e;2 SD). All patients presented some dehydration (mild to moderate), and required no intravenous hydration therapy. Differences in clinical characteristics were not significant among groups.</p><table-wrap position="float" id="T2"><label>Table 2</label><caption><p>Clinical Characteristics of Patients on Admission</p></caption><table frame="hsides" rules="groups"><thead><tr><td></td><td align="left" colspan="2">Apple Juice n = 30</td><td align="center" colspan="2">White Grape Juice n = 30</td><td align="center" colspan="2">Water* n= 30</td></tr><tr><td></td><td colspan="6"><hr></hr></td></tr><tr><td></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center">SD</td></tr></thead><tbody><tr><td align="left">Age (months)</td><td align="left">10.27</td><td align="center">4.75</td><td align="center">10.27</td><td align="center">4.14</td><td align="center">11.09</td><td align="center">4.00</td></tr><tr><td align="left">Diarrhea Duration (hr)</td><td align="left">56.37</td><td align="center">33.90</td><td align="center">64.17</td><td align="center">38.95</td><td align="center">53.47</td><td align="center">33.35</td></tr><tr><td align="left">Fever Duration (hr)</td><td align="left">37.77</td><td align="center">33.46</td><td align="center">47.11</td><td align="center">49.62</td><td align="center">41.04</td><td align="center">38.11</td></tr><tr><td align="left">Vomiting (h)</td><td align="left">41.57</td><td align="center">35.45</td><td align="center">46.97</td><td align="center">39.54</td><td align="center">44.33</td><td align="center">31.44</td></tr><tr><td align="left">Breastfeeding</td><td align="left">13 (n)</td><td align="center">43.3%</td><td align="center">16 (n)</td><td align="center">53.3%</td><td align="center">09 (n)</td><td align="center">30.0 %</td></tr><tr><td align="left">Well Nourished</td><td align="left">16 (n)</td><td align="center">53.3%</td><td align="center">15 (n)</td><td align="center">50.0%</td><td align="center">14 (n)</td><td align="center">46.6%</td></tr><tr><td align="left">Nutritional Risk**</td><td align="left">08(n)</td><td align="center">26.6%</td><td align="center">10(n)</td><td align="center">33.3%</td><td align="center">15(n)</td><td align="center">50.0%</td></tr><tr><td align="left">Mild Malnutrition</td><td align="left">03 (n)</td><td align="center">10.0%</td><td align="center">05(n)</td><td align="center">16.6%</td><td align="center">01(n)</td><td align="center">3.4%</td></tr><tr><td align="left">Moderate/Severe Malnutrition</td><td align="left">03(n)</td><td align="center">10.0%</td><td align="center">00(n)</td><td align="center">0.0%</td><td align="center">00(n)</td><td align="center">0.0%</td></tr></tbody></table><table-wrap-foot><p>There were no statistically significant differences among groups, Analysis of Variance (ANOVA) p &#x0003e; 0.05. Fever was considered above 37.5 C temperature. Nutritional risk indicated body weight deficit &#x0003c; 1 SD, Mild malnutrition &#x0003c; 2 SD and Moderate/Severe Malnutrition &#x0003e; 2 SD.</p><p>n = number of patients and % of patients in each category.</p><p>*Colored/flavored to resemble juice</p></table-wrap-foot></table-wrap><table-wrap position="float" id="T3"><label>Table 3</label><caption><p>Laboratory Data of Patients on Admission*</p></caption><table frame="hsides" rules="groups"><thead><tr><td></td><td align="center" colspan="2">Apple Juice n = 30</td><td align="center" colspan="2">White Grape Juice n = 30</td><td align="center" colspan="2">Water<sup>*</sup> n = 30</td></tr><tr><td></td><td colspan="6"><hr></hr></td></tr><tr><td></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td></tr></thead><tbody><tr><td align="left">Serum Sodium (mEq/l)</td><td align="center">142.03</td><td align="center">5.45</td><td align="center">140.96</td><td align="center">3.51</td><td align="center">140.51</td><td align="center">5.39</td></tr><tr><td align="left">Serum Potassium (mEq/l)</td><td align="center">3.99</td><td align="center">0.59</td><td align="center">3.95</td><td align="center">0.81</td><td align="center">4.27</td><td align="center">0.78</td></tr><tr><td align="left">Hematocrit (%)</td><td align="center">31.17</td><td align="center">3.29</td><td align="center">30.93</td><td align="center">2.50</td><td align="center">32.03</td><td align="center">3.55</td></tr><tr><td align="left">Hemoglobin (g/dl)</td><td align="center">10.28</td><td align="center">1.13</td><td align="center">10.24</td><td align="center">0.87</td><td align="center">10.58</td><td align="center">1.19</td></tr><tr><td align="left">No Anemia</td><td align="center">06(n)</td><td align="center">20.0 %</td><td align="center">02(n)</td><td align="center">6.7 %</td><td align="center">07(n)</td><td align="center">23.3 %</td></tr><tr><td align="left">Mild Anemia &#x02020;</td><td align="center">20(n)</td><td align="center">66.7 %</td><td align="center">26(n)</td><td align="center">86.6%</td><td align="center">20(n)</td><td align="center">66.7 %</td></tr><tr><td align="left">Severe Anemia&#x02020;</td><td align="center">04(n)</td><td align="center">13.3 %</td><td align="center">02(n)</td><td align="center">6.7 %</td><td align="center">03(n)</td><td align="center">10.0 %</td></tr><tr><td colspan="7"><hr></hr></td></tr><tr><td align="left">Rotavirus</td><td align="center">19(n)</td><td align="center">63.3 %</td><td align="center">18(n)</td><td align="center">60.0 %</td><td align="center">18(n)</td><td align="center">60.0 %</td></tr><tr><td align="left">Parasites&#x02021;</td><td align="center">05(n)</td><td align="center">16.7%.</td><td align="center">04(n)</td><td align="center">13.3 %</td><td align="center">01(n)</td><td align="center">3.3 %</td></tr></tbody></table><table-wrap-foot><p>*There were no significant differences among groups by ANOVA p &#x0003e; 0.05.</p><p>&#x02020; The criteria for mild Anemia was a hemoglobin less than 11.0 g/dl and for severe anemia was a hemoglobin less than 9 g/dl (WHO, 1989)</p><p>&#x02021; The parasites detected were: Ascaris lumbricoides (4), Giardia lamblia (1) Blastocytes hominis (1) Entamoeba coli (1) and Cryptosporidium parvum (3).</p><p>n = number of patients and % of patients in each category</p><p><sup>x</sup>Coloured/flavoured to resemble juice</p></table-wrap-foot></table-wrap><p>The serum electrolyte levels on admission to the hospital were similar among the 3 groups of patients (Table <xref ref-type="table" rid="T3">3</xref>). The hemoglobin (Hb) and hematocrit levels were also similar among groups, but two thirds of the patients exhibited mild degrees of anemia (Hb &#x0003c; 11 g), and in 9 infants there was a more severe degree (Hb &#x0003c; 9 g). In all instances iron supplementation was prescribed at the completion of the study. Rotavirus was identified in the stools of 55 of the patients, in 4 there was a pathogenic Escherichia Coli, in 10 there were parasites detected, and in the remaining 21 infants there were no stool pathogens identified.</p><p>The daily intake of the patients while in the study is shown in table <xref ref-type="table" rid="T4">4</xref>. The amount of water, milk formula, and breast milk feedings did not differ among the groups. However, the infants given WGJ readily consumed more juice than those fed AJ or WA. The total energy intake was higher in the juice fed groups as compared with the WA one. The WGJ patients ingested an average of 17% more calories on a daily basis, and the AJ infants consumed a mean of 14% more than the WA group. The increased energy intake was not at the expense of the other foods; both milk formula and complementary foods were ingested in similar quantities among the 3 groups of patients. The mean body weight gain was also higher among the juice fed patients; there was a mean weight gain of 18.3 gm in the AJ fed patients and of 50.6 gm in those given WGJ, whereas there was a mean loss of body weight (- 7.0 gm) among the WA group patients (p = 0.08).</p><table-wrap position="float" id="T4"><label>Table 4</label><caption><p>Daily intake of patients throughout the study (Kcal/Kg/day)</p></caption><table frame="hsides" rules="groups"><thead><tr><td></td><td align="center" colspan="2">Apple Juice n = 30</td><td align="center" colspan="2">White Grape Juice n = 30</td><td align="center" colspan="2">Water<sup>*</sup> n = 30</td><td align="center">P value</td></tr><tr><td></td><td colspan="6"><hr></hr></td><td></td></tr><tr><td></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td align="center">&#x003c7;</td><td align="center"><italic>SD</italic></td><td></td></tr></thead><tbody><tr><td align="left">Total Calories</td><td align="center">95.84</td><td align="center">22.42</td><td align="center">98.65</td><td align="center">30.52</td><td align="center">81.43</td><td align="center">23.09</td><td align="center">0.02*</td></tr><tr><td align="left">Milk Formula</td><td align="center">54.49</td><td align="center">23.43</td><td align="center">50.21</td><td align="center">34.57</td><td align="center">52.68</td><td align="center">17.93</td><td align="center">0.81</td></tr><tr><td align="left">Breast Milk</td><td align="center">08.71</td><td align="center">11.57</td><td align="center">15.80</td><td align="center">18.56</td><td align="center">05.45</td><td align="center">10.96</td><td align="center">0.06</td></tr><tr><td align="left">Water</td><td align="center">30.67</td><td align="center">9.25</td><td align="center">30.27</td><td align="center">09.57</td><td align="center">26.98</td><td align="center">9.37</td><td align="center">0.25</td></tr><tr><td align="left">ORS</td><td align="center">45.52</td><td align="center">31.17</td><td align="center">39.12</td><td align="center">25.10</td><td align="center">25.10</td><td align="center">17.91</td><td align="center">0.01**</td></tr><tr><td align="left">"Juices"</td><td align="center">18.61</td><td align="center">3.93</td><td align="center">20.98</td><td align="center">5.35</td><td align="center">17.32<sup>x</sup></td><td align="center">4.37</td><td align="center">0.01***</td></tr><tr><td align="left">Total Liquids</td><td align="center">157.90</td><td align="center">38.26</td><td align="center">158.42</td><td align="center">50.35</td><td align="center">127.70</td><td align="center">28.25</td><td align="center">0.001****</td></tr><tr><td align="left">Complementary Foods</td><td align="center">23.90</td><td align="center">11.61</td><td align="center">25.12</td><td align="center">13.97</td><td align="center">26.92</td><td align="center">11.59</td><td align="center">0.64</td></tr></tbody></table><table-wrap-foot><p>* Significant differences for Juice Groups vs Water, by ANOVA (Bonferroni)</p><p>** Significant differences for Water vs both juice groups</p><p>*** Significant differences for WGJ vs each other group</p><p>**** Significant Differences for Water vs each other group</p><p><sup>x </sup>Colored flavored water to resemble juice</p><p>Data are means +/- SD</p></table-wrap-foot></table-wrap><p>The duration of diarrhea differed among the 3 groups of patients (Table <xref ref-type="table" rid="T5">5</xref>). The total duration of diarrhea from the start of the illness through their recovery was decreased among the WA fed patients as compared with the AJ and WGJ: groups 111.7 &#x000b1; 48.2, 105.4 &#x000b1; 44.9 and 80.0 &#x000b1; 39.6 hours in AJ, WGJ and WA respectively. The differences in the duration of diarrhea were more marked while being treated in the hospital, the illness being shorter among patients given water instead of juice. However the majority of the patients recovered promptly regardless of the treatment given (Figure <xref ref-type="fig" rid="F1">1</xref>). Most of the patients improved within 50 hours after treatment was instituted, less than one forth of them had diarrhea persisting more than 96 hours and no one had it for more than 7 days.</p><table-wrap position="float" id="T5"><label>Table 5</label><caption><p>Duration of diarrhea in hours after randomization. Duration of diarrhea</p></caption><table frame="hsides" rules="groups"><thead><tr><td></td><td align="left">Mean</td><td align="left">SD</td></tr></thead><tbody><tr><td align="left">Apple Juice</td><td align="left">49.4</td><td align="left">32.6</td></tr><tr><td align="left">White Grape Juice</td><td align="left">47.5</td><td align="left">38.9</td></tr><tr><td align="left">Water *</td><td align="left">26.5</td><td align="left">27.4</td></tr></tbody></table><table-wrap-foot><p>*Significant differences detect by Kruskal-Wallis p &#x0003c; 0.05. Water vs Juice groups. Data are hours +/- SD</p><p>Water coloured and flavored to resemble juice.</p></table-wrap-foot></table-wrap><fig position="float" id="F1"><label>Figure 1</label><caption><p>Survival analysis of total of diarrhea &#x02013; Kaplan-Meier, per group. * Statistic difference were found, p &#x0003c; 0,05, among water group and apple juice (P = 0,03) or white grape juice (P = 0,00).</p></caption><graphic xlink:href="1475-2891-4-23-1"></graphic></fig><p>The severity of diarrhea also differed among the treatment groups (Table <xref ref-type="table" rid="T6">6</xref>). The stool output among the WA group of patients was significantly decreased as compared with those fed juice. During the first day of treatment those fed water had a mean stool output of 40% less than those fed juice. With improvement the differences among the treatments were minimized, and were no longer significant after the 2<sup>nd </sup>day of the illness. There were also significant differences in the severity of diarrhea among AJ and WGJ feedings (Table <xref ref-type="table" rid="T6">6</xref>). During the first 24 hours of treatment of the illness the patients receiving AJ had more marked stool losses than those fed WGJ. The mean excretion of stools was 21% higher in AJ fed patients than in the WGJ fed group.</p><table-wrap position="float" id="T6"><label>Table 6</label><caption><p>Fecal losses throughout the study and on the first day after randomization</p></caption><table frame="hsides" rules="groups"><thead><tr><td></td><td align="center" colspan="2"><bold>Total Losses (g/kg/hr)</bold></td><td align="center" colspan="2"><bold>First Day Losses g/kg/hr</bold></td></tr></thead><tbody><tr><td></td><td align="center">Mean</td><td align="center">SD</td><td align="center">Mean</td><td align="center">SD</td></tr><tr><td colspan="5"><hr></hr></td></tr><tr><td align="left">Apple Juice</td><td align="center">3.94</td><td align="center">2.35</td><td align="center">4.13**</td><td align="center">2.90</td></tr><tr><td align="left">White Grape Juice</td><td align="center">3.59</td><td align="center">2.35</td><td align="center">3.28**</td><td align="center">2.39</td></tr><tr><td align="left">Water<sup>x</sup></td><td align="center">2.19*</td><td align="center">1.63</td><td align="center">1.78***</td><td align="center">1.80</td></tr></tbody></table><table-wrap-foot><p>*Differences Among groups Water vs each of the juice groups (Kruskal-Wallis test) (p = 0.001)</p><p>** Differences between WGJ and AJ (p = 0.02)</p><p>*** Differences between Water and each of the juice groups (p = 0.001)</p><p><sup>x</sup>Coloured/flavoured to resemble juice</p></table-wrap-foot></table-wrap><p>The stool excretion data were treated for possible confounding and co-variables which could play a role in determining the final results. The covariance analysis and the robust regression showed a possible influence of ORS intake in the differences detected among the 3 groups of patients. The stool output was not different when the ORS intake was adjusted for the 3 groups of patients and the urinary outputs were also similar. Vomitus losses were not different among the 3 groups of patients, vomiting being present primarily during the first 24 hrs. of treatment.</p><p>There were 2 patients who had severe diarrhea during the treatment period (&#x0003e;10 ml/kg/hr). These patients were in the AJ group, one of these patients had acid stools and none had carbohydrates in feces. These 2 patients were treated with a lactose free formula, with improvement in stool output. There were three other patients who showed carbohydrate intolerance with acid stools or sugars in feces among the 3 groups, they improved without any dietary modifications.</p><p>Vomiting was present in all groups of patients: 22 (AJ), 26 (WGJ), and 19 (WA) during the first day of treatment; nevertheless this did not represented a limitation for Oral Rehydration Therapy (ORT) or feeding regimen and no patient was withdrawn or shifted to another therapy due to vomiting.</p><p>The response to juice feedings, as determined by breath hydrogen excretion after improvement of the illness, was not different among the 3 groups of patients. Most infants did not show breath hydrogen excretion levels above 20 ppm; and there were 17, 16 and 10 patients among the AJ, WGJ and WA groups respectively, who failed to show BH2 levels above 5 ppm at any time. Only 18 patients demonstrated a delta BH2 level increase above the basal value of more than 20 ppm. Eight of them were given AJ, 6 were fed WGJ and 4 received WA. There were 11, 16, and 13 patients in each group who exhibited a BH2 value above 20 ppm independent of the delta differences from basal among the WGJ, AJ and WA fed groups respectively. The differences in BH2 levels among groups were not significant.</p></sec>
"""
prompt = continuous_outcomes_prompt(article, "White Grape Juice (WGJ)", "colored and flavored water (WA)", "The duration of the illness")
generate(model, tokenizer, prompt, max_new_tokens=100)

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.67 GiB. GPU 0 has a total capacity of 44.48 GiB of which 3.17 GiB is free. Including non-PyTorch memory, this process has 41.31 GiB memory in use. Of the allocated memory 37.93 GiB is allocated by PyTorch, and 3.19 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# NEXT STEPS:

# 1. trying more examples from dev split (specifically articles with longer tokens)
# 2. trying different prompts to get better results and also consistent format (e.g. json output format)